# Продвинутый Python, семинар 2 

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Думая о том, где можно было бы хорошо попрактиковать анализ данных, меня вдруг осенило, что одно из явных приложений - это аналитика игр

Никакая успешная команда, выступающая на чемпионатах (будь то The International в Dota2 или же Worlds в LoL), всегда должна иметь аналитика, который будет анализировать игры и на их основе находить инсайты по тому, какую композицию выбрать или же различные синергии.

Поэтому мы будем анализировать игру, которая очень нравится самому лектору - TFT (TeamFight Tactics). Это автобаттлер, в котором ты размещаешь фигурки, апгейдишь их, кладешь итемы и так далее.

![TFT](https://genapilot.ru/wp-content/uploads/2020/03/teamfight-tactics-vyhodit-na-mobilnyh-ustrojstvah-s-ios-i-android.jpg)

Поэтому попрактикуемся на базовом датасете: [тык](https://www.kaggle.com/datasets/pvillanueva13/tft-35-500-gm-matches?select=tft_3.5_last_500_gm_match_info.csv)

## Грузим данные и библиотеки

In [ ]:
!pip install python-dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.parser import parse
import dateutil
import pytz

In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: items.json              
  inflating: tft_3.5_last_500_gm_match_info.csv  
  inflating: tft_3.5_last_500_gm_unit_info.csv  
  inflating: traits.json             


## Что у нас есть?

Наши данные - это информация о 500 матчах игроков и фигурках, которые были поставлены.

А также jsonы с информацией про предметы и особенности

### Задание 1.

Загрузите данные и посмотрите на них

Отлично! Что мы видим:

1. Куча каких-то непонятных id

2. Поля, которые нам мало что говорят (если вы только не заядлый игрок в TFT)

3. И NaN, как же без них

## Задание 2.

Посчитайте число строк в датасете matches и иосмотрите на число уникальных puuid и match_id в датасете с матчами.

Сделайте вывод, сколько было матчей и сколько игроков играло

## Задание 3.

Определить, сколько было челленджеров (это люди, у которых самый высокий ранг, столбец challenger)

## Задание 4.

Посмотреть, сколько участников было в каждом матче

## Задание 5.

Создайте колонку winner, где 1-4 место - это победа, а 5-8 - проигрыш

А также создайте колонку galaxy_numb, где каждой галактике будет присвоен свой собственный номер

И после этого посмотреть на распределение мест у challengers и распределение галактик в играх

Видим, что игроки с высоким рейтингом очень часто выигрывают!

## Datetime

Чтобы решать задачи со временем, можно было бы просто вычленить из строки нужное поле и жить спокойно. Но вот, допустим, что вы хотите посмотреть не только час, но и минута, день, месяц, год и так далее. А еще у вас там часовой пояс есть какой-нибудь.

Выглядит не очень. Вот ровно для такого и существует библиотека datetime! С ее помощью можно достать из строки дату и время, преобразовать ее, привести к общему часовому поясу, посчитать разницу во времени и так далее

[Документация](https://docs.python.org/3/library/datetime.html)

Есть еще полезная надстройка над datetime - dateutil, которая в некоторых моментах позволяет более удобно работать со временем и его преобразованием

[Документация](https://dateutil.readthedocs.io/en/stable/index.html)

Какие функции есть у datetime? Начнем с преобразований из строк

* datetime.fromisoformat(s) - работает, если у вас формат выглядит как

```
YYYY-MM-DD[*HH[:MM[:SS[.fff[fff]]]][+HH:MM[:SS[.ffffff]]]]
YYY-MM-DD - год-месяц-день
HH:MM:SS - часы-минуты-секунды (может быть еще после точки миллисекунды)
+HH::MM:SS - часовой пояс не через название
```

* datetime.strptime(s, format) - по заданному шаблону преобразовать в datetime (однако тут надо задать правильный шаблон)

Форматы:

```
%a  Locale’s abbreviated weekday name.
%A  Locale’s full weekday name.      
%b  Locale’s abbreviated month name.     
%B  Locale’s full month name.
%c  Locale’s appropriate date and time representation.   
%d  Day of the month as a decimal number [01,31].    
%f  Microsecond as a decimal number [0,999999], zero-padded on the left
%H  Hour (24-hour clock) as a decimal number [00,23].    
%I  Hour (12-hour clock) as a decimal number [01,12].    
%j  Day of the year as a decimal number [001,366].   
%m  Month as a decimal number [01,12].   
%M  Minute as a decimal number [00,59].      
%p  Locale’s equivalent of either AM or PM.
%S  Second as a decimal number [00,61].
%U  Week number of the year (Sunday as the first day of the week)
%w  Weekday as a decimal number [0(Sunday),6].   
%W  Week number of the year (Monday as the first day of the week)
%x  Locale’s appropriate date representation.    
%X  Locale’s appropriate time representation.    
%y  Year without century as a decimal number [00,99].    
%Y  Year with century as a decimal number.   
%z  UTC offset in the form +HHMM or -HHMM.
%Z  Time zone name (empty string if the object is naive).    
%%  A literal '%' character.
```

* dateutil.parser.parse(s) - спарсить в datetime (без лишних заморочек с форматом)

In [ ]:
d = '2022-04-26T15:14:32.001+03:00'

dt.datetime.fromisoformat(d)

datetime.datetime(2022, 4, 26, 15, 14, 32, 1000, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))

In [ ]:
from dateutil.parser import parse
#А теперь поменяем: DD-MM-YYYY
d = '05-04-2022 15:14:32.001GMT'
format = '%d-%m-%Y %H:%M:%S.%f%Z'

dt.datetime.strptime(d, format)

datetime.datetime(2022, 4, 5, 15, 14, 32, 1000)

In [ ]:
print(parse(d, dayfirst=True)) #Только здесь надо было указать, что день идет первее, чем месяц

tzinfos = {'MSK': 10800}
d = 'Sat Oct 11 17:13:46 MSK 2003'
print(parse(d, tzinfos=tzinfos)) # И даже так он умеет, но надо задавать все нужные вам таймзоны

2003-10-11 17:13:46+03:00


In [ ]:
dt.datetime.fromtimestamp(17456034) #можно вытаскивать также и с timestamp время

datetime.datetime(1970, 7, 22, 0, 53, 54)

Мы получили инстанс в виде datetime, из которого можно спокойно вытащить примерно все:

In [ ]:
parse(d, tzinfos=tzinfos)

datetime.datetime(2003, 10, 11, 17, 13, 46, tzinfo=tzoffset('MSK', 10800))

In [ ]:
d = parse(d, tzinfos=tzinfos)
print(d.year, d.month, d.day, d.hour, d.minute, d.second, d.tzinfo)

2003 10 11 17 13 46 tzoffset('MSK', 10800)


Окей, теперь хотим избавиться от тайм-зоны (или привести к какой-нибудь другой зоне, скажем, МСК подойдет)

In [ ]:
d = d.astimezone(pytz.timezone('UTC')) 
print(d)
d = d.astimezone(pytz.timezone('Europe/Moscow')) 
print(d)

2003-10-11 14:13:46+00:00
2003-10-11 18:13:46+04:00


In [ ]:
#Вывести все возможные таймзоны
for tz in pytz.all_timezones:
    print(tz)

В чем проблема с таймзонами? В том, что с ними в целом сложно (вот у нас был переход на зимнее/летнее время, теперь нет, а потом опять ввели, а потом опять нет)

Поэтому такие косяки бывают всегда, но нам для точности достаточно хотя бы привести все к единому времени (так у нас данные не будут расходиться по часовым поясам)

Хорошо, привели все времена. Хотим теперь, наверное, считать diff между временами? Легко!

In [ ]:
dt_1 = dt.datetime(2022, 5, 6, 14, 15, 16)
print(dt_1)
dt_2 = dt.datetime.now() #Дает время в UTC
print(dt_2)

2022-05-06 14:15:16
2022-09-14 22:50:31.705200


In [ ]:
dt_2 - dt_1 #Получаем объект типа timedelta

datetime.timedelta(days=131, seconds=30915, microseconds=705200)

In [ ]:
diff = dt_2 - dt_1
diff.days #хопа, разница в днях

131

In [ ]:
dt_2 + dt.timedelta(days=15) #Хотим узнать, какое число будет через 15 дней

datetime.datetime(2022, 9, 29, 22, 50, 31, 705200)

In [ ]:
format = '%d-%m-%Y'
dt.datetime.strftime(dt_2 + dt.timedelta(days=15), format) #и красивый вывод по шаблону

'29-09-2022'

А теперь к заданию!

## Задание 6.

Понять, в какие часы и дни играют challengers и выигрывают чаще. Для этого нужно создать поля days и hours с помощью datetime :)

## Задание 7.

Сджойнить таблицы с героями и матчами и выбрать самого лучшего чемпиона (самое больше число побед)

## Задание 8.

Поменяйте в таблице с героями все Nan на 0, а затем посчитайте и добавьте индекс "полезности" сборки (индекс будем считать как скалярное произведение id итемов на столбец [2, 1, 0] (то есть первый предмет самый нужный, затем второй и только после третий)

Добавьте колонку "coef" в датасет

## Задание 9.

Посчитать, какие итемы чаще всего используют и как они называются

Иногда для того, чтобы выгрузить данные, вам надо будет подгружать json-файлы. Делается это с помощью библиотеки json

[Документация](https://docs.python.org/3/library/json.html) 

Базово, для того, чтобы выгрузить json, нужно использовать load (когда грузим из файла) или loads (если хотим выгрузить из строки)

In [ ]:
import json

with open('items.json') as json_data:
    items = json.load(json_data)
items = pd.DataFrame(items) # А дальше можно просто запихнуть это в DataFrame, очень удобно!

Логичнее, наверное, вначале предрасчитать, а потом это просто добавить к item_id!

## Задание 10.

Удалите поля со всеми нецелочисленными колонками.

Запишите получившийся результат для матчей в matches_new.csv

![Жако](https://petshop-vrn.ru/wp-content/uploads/6/c/a/6ca4dd99334ae553c70fbc4ed4165506.jpeg)

А это Жако, или серый попугай. Считаются самыми умными попугаями в мире, потому что они способны не просто на подражание звуков (как это делают многие попугаи)

В 1980-х годах ученая Ирэн Пепперберг проводила исследования по языковому мышлению различных животных, и одним из подопытных был серый попугай по имени Алекс. По результатам эксперимента Алекс мог произносить очень много слов, причем он верно отвечал на вопросы и в целом коммуницировал исходя не из подражания, а из логики. В целом было показано, что у него было мышление, как у шестилетнего ребенка, что стало полным открытием для ученых.

Хотя, на самом деле, сравнивать мышление попугаев и людей сложно, потому что мышление животных и не должно быть похожим на человеческое